# 🎬 YouTube 视频转录工具

将 YouTube 视频转换为文字和字幕文件 (txt + srt)

**使用步骤：**
1. 📁 运行「挂载 Google Drive」
2. 📦 运行「安装依赖」
3. ⚙️ 填写配置（YouTube URL、模型等）
4. 🚀 运行「执行转录」

> ⚠️ 如遇 403 错误，请查看底部「常见问题」部分

In [1]:
#@title 📁 挂载 Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 设置输出目录
import os
OUTPUT_DIR = "/content/drive/MyDrive/YouTubeTranscripts"
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"✅ 输出目录: {OUTPUT_DIR}")

# 检查 cookies 文件
COOKIES_PATH = "/content/drive/MyDrive/cookies.txt"
if os.path.exists(COOKIES_PATH):
    print(f"🍪 发现 cookies 文件: {COOKIES_PATH}")
else:
    print("ℹ️ 未发现 cookies.txt，如遇 403 错误请参考说明上传")

Mounted at /content/drive
✅ 输出目录: /content/drive/MyDrive/YouTubeTranscripts
ℹ️ 未发现 cookies.txt，如遇 403 错误请参考说明上传


In [ ]:
#@title 📦 安装依赖 (自动检查，已安装则跳过)

import subprocess
import sys
import os

def check_and_install(package_name, import_name=None):
    """检查包是否已安装，未安装则安装"""
    import_name = import_name or package_name
    try:
        __import__(import_name)
        print(f"✅ {package_name} 已安装")
        return True
    except ImportError:
        print(f"📦 正在安装 {package_name}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package_name])
        print(f"✅ {package_name} 安装完成")
        return False

# 安装 Deno JavaScript 运行时 (yt-dlp 2025+ 必需)
print("📦 检查/安装 Deno JavaScript 运行时...")
deno_path = os.path.expanduser("~/.deno/bin/deno")
if not os.path.exists(deno_path):
    subprocess.run("curl -fsSL https://deno.land/install.sh | sh", shell=True, check=True)
    print("✅ Deno 安装完成")
else:
    print("✅ Deno 已安装")

# 添加 Deno 到 PATH
os.environ["PATH"] = os.path.expanduser("~/.deno/bin") + os.pathsep + os.environ.get("PATH", "")

# 强制更新 yt-dlp 到最新版本 (解决 403 问题的关键)
print("📦 更新 yt-dlp 到最新版本...")
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "--upgrade", "yt-dlp"])
print("✅ yt-dlp 已更新")

# 检查并安装 whisper
check_and_install("openai-whisper", "whisper")

print("\n✅ 所有依赖就绪！")

📦 更新 yt-dlp 到最新版本...
✅ yt-dlp 已更新
📦 正在安装 openai-whisper...
✅ openai-whisper 安装完成

✅ 所有依赖就绪！


In [3]:
#@title ⚙️ 配置

#@markdown ### 必填：YouTube 视频 URL
YOUTUBE_URL = "https://youtu.be/6W6BB18tpnE?si=pQgMA_za1PQOdFaq" #@param {type:"string"}

#@markdown ### 可选设置
MODEL = "large-v3" #@param ["tiny", "base", "small", "medium", "large", "large-v2", "large-v3"]
LANGUAGE = "" #@param {type:"string"}
#@markdown 留空自动检测语言，或指定语言代码如: `zh`, `en`, `ja`

#@markdown ### 403 错误修复
USE_COOKIES = False #@param {type:"boolean"}
#@markdown 启用后将使用 Google Drive 根目录的 cookies.txt

print(f"🎬 URL: {YOUTUBE_URL}")
print(f"🤖 模型: {MODEL}")
print(f"🌐 语言: {LANGUAGE if LANGUAGE else '自动检测'}")
print(f"🍪 使用 Cookies: {'是' if USE_COOKIES else '否'}")

🎬 URL: https://youtu.be/6W6BB18tpnE?si=pQgMA_za1PQOdFaq
🤖 模型: large-v3
🌐 语言: 自动检测
🍪 使用 Cookies: 否


In [ ]:
#@title 🚀 执行转录

import re
import tempfile
import time
from pathlib import Path
import yt_dlp
import whisper

def parse_youtube_url(url: str) -> str | None:
    """解析 YouTube URL，提取 video_id"""
    patterns = [
        r'(?:https?://)?(?:www\.)?youtube\.com/watch\?v=([a-zA-Z0-9_-]{11})',
        r'(?:https?://)?(?:www\.)?youtube\.com/embed/([a-zA-Z0-9_-]{11})',
        r'(?:https?://)?(?:www\.)?youtube\.com/v/([a-zA-Z0-9_-]{11})',
        r'(?:https?://)?youtu\.be/([a-zA-Z0-9_-]{11})',
    ]
    for pattern in patterns:
        match = re.search(pattern, url)
        if match:
            return match.group(1)
    return None

def sanitize_filename(filename: str, max_bytes: int = 180) -> str:
    """清理文件名，移除非法字符，并限制字节长度（兼容中文）"""
    illegal_chars = r'[<>:"/\\|?*]'
    sanitized = re.sub(illegal_chars, '_', filename)

    # 按字节长度截断（UTF-8 中文占 3 字节，留余量给扩展名）
    encoded = sanitized.encode('utf-8')
    if len(encoded) > max_bytes:
        # 逐字符截断，确保不截断在字符中间
        result = ''
        current_bytes = 0
        for char in sanitized:
            char_bytes = len(char.encode('utf-8'))
            if current_bytes + char_bytes > max_bytes:
                break
            result += char
            current_bytes += char_bytes
        return result
    return sanitized

def format_timestamp(seconds: float) -> str:
    """将秒数转换为 SRT 时间戳格式 (HH:MM:SS,mmm)"""
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    secs = int(seconds % 60)
    millis = int((seconds % 1) * 1000)
    return f"{hours:02d}:{minutes:02d}:{secs:02d},{millis:03d}"

def format_duration(seconds: float) -> str:
    """将秒数格式化为可读时长 (MM:SS 或 HH:MM:SS)"""
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    secs = int(seconds % 60)
    if hours > 0:
        return f"{hours:02d}:{minutes:02d}:{secs:02d}"
    return f"{minutes:02d}:{secs:02d}"

def download_audio(url: str, output_dir: Path, use_cookies: bool = False) -> tuple:
    """使用 yt-dlp 下载音频，尝试多种客户端绕过 403"""
    print("📥 正在下载音频...")

    cookies_path = "/content/drive/MyDrive/cookies.txt"
    has_cookies = use_cookies and Path(cookies_path).exists()

    # 基础配置
    base_opts = {'quiet': True}
    if has_cookies:
        base_opts['cookiefile'] = cookies_path
        print(f"🍪 使用 cookies: {cookies_path}")

    # 获取视频信息
    with yt_dlp.YoutubeDL(base_opts) as ydl:
        info = ydl.extract_info(url, download=False)
        video_title = info.get('title', 'unknown')

    safe_title = sanitize_filename(video_title)
    output_template = str(output_dir / f"{safe_title}.%(ext)s")

    # 尝试多种客户端配置来绕过 403
    client_configs = [
        # 优先尝试 iOS 客户端（通常能绕过 SABR 限制）
        {'player_client': 'ios,web'},
        # 尝试 Android 客户端
        {'player_client': 'android,web'},
        # 尝试 TV 客户端
        {'player_client': 'tv,web'},
        # 默认配置
        {},
    ]

    last_error = None
    for i, extractor_args in enumerate(client_configs):
        client_name = extractor_args.get('player_client', 'default')
        print(f"🔄 尝试客户端: {client_name}")

        ydl_opts = {
            'format': 'bestaudio/best',
            'outtmpl': output_template,
            'postprocessors': [{
                'key': 'FFmpegExtractAudio',
                'preferredcodec': 'mp3',
                'preferredquality': '192',
            }],
            'quiet': False,
            'no_warnings': False,
        }

        if extractor_args:
            ydl_opts['extractor_args'] = {'youtube': extractor_args}

        if has_cookies:
            ydl_opts['cookiefile'] = cookies_path

        try:
            with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                ydl.download([url])

            audio_path = output_dir / f"{safe_title}.mp3"
            if audio_path.exists():
                print(f"✅ 音频下载完成: {audio_path.name}")
                return audio_path, video_title
        except Exception as e:
            last_error = e
            print(f"⚠️ 客户端 {client_name} 失败: {str(e)[:100]}")
            continue

    # 所有方法都失败
    raise Exception(f"❌ 所有下载方式都失败。最后错误: {last_error}\n\n"
                    "💡 建议：请启用 Cookies 功能（参考底部说明上传 cookies.txt）")

def transcribe_audio(audio_path: Path, model_name: str, language: str = None) -> dict:
    """使用 Whisper 转录音频，并显示处理速度"""
    print(f"🔄 正在加载 Whisper 模型 ({model_name})...")
    model = whisper.load_model(model_name)

    # 获取音频时长
    from whisper.audio import load_audio, SAMPLE_RATE
    audio = load_audio(str(audio_path))
    audio_duration = len(audio) / SAMPLE_RATE
    print(f"📊 音频时长: {format_duration(audio_duration)}")

    print("🎙️ 正在转录音频...")
    print("-" * 60)

    # 自定义回调显示进度
    transcribe_options = {'fp16': True, 'verbose': False}
    if language:
        transcribe_options['language'] = language

    start_time = time.time()
    result = model.transcribe(str(audio_path), **transcribe_options)
    total_elapsed = time.time() - start_time

    # 显示每个片段及处理速度
    segments = result.get('segments', [])
    for i, segment in enumerate(segments):
        seg_start = segment['start']
        seg_end = segment['end']
        text = segment['text'].strip()

        # 计算当前进度和速度
        progress = (seg_end / audio_duration) * 100 if audio_duration > 0 else 0

        # 显示片段信息
        timestamp = f"[{format_duration(seg_start)} -> {format_duration(seg_end)}]"
        print(f"{timestamp} {text}")

    print("-" * 60)

    # 显示总体统计
    speed_ratio = audio_duration / total_elapsed if total_elapsed > 0 else 0
    detected_lang = result.get('language', 'unknown')

    print(f"✅ 转录完成！")
    print(f"   🌐 检测语言: {detected_lang}")
    print(f"   ⏱️ 音频时长: {format_duration(audio_duration)}")
    print(f"   ⚡ 处理耗时: {format_duration(total_elapsed)}")
    print(f"   🚀 处理速度: {speed_ratio:.1f}x 实时速度")

    return result

def save_outputs(result: dict, output_dir: str, title: str) -> tuple:
    """保存 txt 和 srt 文件到 Google Drive"""
    safe_title = sanitize_filename(title)
    output_path = Path(output_dir)

    # 保存 txt
    txt_file = output_path / f"{safe_title}.txt"
    with open(txt_file, 'w', encoding='utf-8') as f:
        f.write(result['text'].strip())
    print(f"📄 文本文件已保存: {txt_file}")

    # 保存 srt
    srt_file = output_path / f"{safe_title}.srt"
    segments = result.get('segments', [])
    with open(srt_file, 'w', encoding='utf-8') as f:
        for i, segment in enumerate(segments, start=1):
            start_time = format_timestamp(segment['start'])
            end_time = format_timestamp(segment['end'])
            text = segment['text'].strip()
            f.write(f"{i}\n{start_time} --> {end_time}\n{text}\n\n")
    print(f"📄 字幕文件已保存: {srt_file}")

    return txt_file, srt_file

def disconnect_runtime():
    """断开 Colab 运行时以节省流量"""
    try:
        from google.colab import runtime
        print("🔌 3 秒后断开运行时...")
        time.sleep(3)
        runtime.unassign()
    except Exception:
        # 非 Colab 环境则忽略
        pass

# ===== 主执行流程 =====
video_id = parse_youtube_url(YOUTUBE_URL)
if not video_id:
    raise ValueError("❌ 无效的 YouTube URL，请检查后重试")

print(f"🎬 Video ID: {video_id}")
print("="*50)

# 创建临时目录
temp_dir = Path(tempfile.mkdtemp())
success = False

try:
    # 下载音频
    audio_path, video_title = download_audio(YOUTUBE_URL, temp_dir, USE_COOKIES)

    # 转录
    language_opt = LANGUAGE if LANGUAGE else None
    result = transcribe_audio(audio_path, MODEL, language_opt)

    # 保存到 Google Drive
    txt_file, srt_file = save_outputs(result, OUTPUT_DIR, video_title)

    print("\n" + "="*50)
    print("✨ 处理完成！")
    print(f"📁 文件位置: {OUTPUT_DIR}")
    print(f"   📄 {txt_file.name}")
    print(f"   📄 {srt_file.name}")
    success = True

except Exception as e:
    print(f"\n❌ 处理失败: {e}")
    success = False

finally:
    # 清理临时文件
    import shutil
    if temp_dir.exists():
        shutil.rmtree(temp_dir, ignore_errors=True)
        print("🧹 临时文件已清理")

    # 无论成功失败都断开运行时节省流量
    disconnect_runtime()

🎬 Video ID: 6W6BB18tpnE
📥 正在下载音频...


[youtube] Extracting URL: https://youtu.be/6W6BB18tpnE?si=pQgMA_za1PQOdFaq
[youtube] 6W6BB18tpnE: Downloading webpage


[youtube] 6W6BB18tpnE: Downloading android sdkless player API JSON
[youtube] 6W6BB18tpnE: Downloading web safari player API JSON


[youtube] 6W6BB18tpnE: Downloading m3u8 information


[info] 6W6BB18tpnE: Downloading 1 format(s): 251


ERROR: unable to download video data: HTTP Error 403: Forbidden


🧹 临时文件已清理


DownloadError: ERROR: unable to download video data: HTTP Error 403: Forbidden

---

## 📋 说明

### Whisper 模型选择
| 模型 | 大小 | 速度 | 准确度 |
|------|------|------|--------|
| tiny | 39M | 最快 | 较低 |
| base | 74M | 快 | 一般 |
| small | 244M | 中等 | 良好 |
| medium | 769M | 较慢 | 很好 |
| large-v3 | 1550M | 慢 | 最佳 |

### 输出文件
- `.txt` - 纯文本转录内容
- `.srt` - 带时间轴的字幕文件，可导入视频播放器

### 文件位置
输出文件保存在: `Google Drive/YouTubeTranscripts/`

---

## 🔧 常见问题

### HTTP 403 Forbidden 错误
这是 YouTube 的反爬虫机制，解决方法：

**方法 1: 使用 Cookies (推荐)**
1. 安装浏览器扩展 [Get cookies.txt LOCALLY](https://chromewebstore.google.com/detail/get-cookiestxt-locally/cclelndahbckbenkjhflpdbgdldlbecc)
2. 登录 YouTube 账号
3. 在 YouTube 页面点击扩展，导出 cookies.txt
4. 上传到 Google Drive 根目录 (MyDrive/cookies.txt)
5. 在配置中设置 `USE_COOKIES = True`

**方法 2: 等待 yt-dlp 更新**
- yt-dlp 会定期更新以适应 YouTube 变化
- 重新运行「安装依赖」单元格会自动更新到最新版

### 视频无法下载
以下视频可能无法下载：
- 私享视频 (Private)
- 年龄限制视频 (需登录 + Cookies)
- 地区限制视频
- 付费/会员专享内容